In [1]:
import astroprov
from astropy.io import fits
from astropy.io.ascii import SExtractor
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os
from photutils import DAOStarFinder
from photutils import CircularAperture
from photutils import aperture_photometry
#import subproc

import numpy as np
from astropy import wcs
from astropy.io import fits
import sys

from astroquery.simbad import Simbad

import matplotlib.pyplot as plt
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table, vstack
from astropy.table import Column

import collections
import subprocess
from astropy.io import ascii
import time
from astropy.utils.data import clear_download_cache

In [2]:
# diffDir = '/media/mj1e16/PP AV-TV/properDiff/'
# medianDir = '/media/mj1e16/PP AV-TV/medians/'
# imageDir = '/media/mj1e16/PP AV-TV/keplerCal/'

In [3]:
diffDir = '/data/mj1e16/kepler/properDiff2/'
medianDir = '/data/mj1e16/kepler/medians/'
imageDir = '/data/mj1e16/kepler/cal/'
sexDir = '/home/mj1e16/sextractor/sextractor-master/config/'

In [4]:
catName = 'test_test.cat' # CATAGLOFUE_NAME IN SEXTRACTOR SETTINGS

In [5]:
clear_download_cache()

In [6]:
def makeConfig(valList,attributeList,tableName='table',defaultDir=sexDir,confName='default_test.sex',catName='test_test.cat'):
    
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData
    
    cname = 'CATALOG_NAME'
    #catName = 'test1.cat'
    #confName = 'default_1.sex'
    nameLoc = data.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' ' +catName+ ' ' + data[endLoc:]
    data = newData

    cname = 'PARAMETERS_NAME'
    nameLoc = data.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' autodefault.param ' + data[endLoc:]
    
    #print(newData)
    with open(defaultDir+confName,'w') as f:
        f.write(newData)
    return(confName,tableName,catName)

In [7]:
def sextractor(sextractoryDir,imagename,cataloguename,imageDir='/home/mj1e16/keplerImages/'):
    
    subprocess.call(['sex',imageDir+imagename,'-c','default_test.sex'])
    assoc = Table.read(cataloguename,format='ascii.sextractor')

#     os.chdir(imageDir)
#     hdu_list = fits.open(imagename) # open image header
#     imagedata = (hdu_list[0].data) 
    
#     fig, ax = plt.subplots(figsize=(10, 10))
#     positions = (assoc['X_IMAGE'], assoc['Y_IMAGE'])
#     apertures = CircularAperture(positions, r=4.)
#     norm = ImageNormalize(stretch=SqrtStretch())
#     ax.imshow(imagedata, cmap='Greys', origin='lower', vmin=-100,vmax=100)#norm=norm)
#     apertures.plot(color='blue', lw=1.5, alpha=0.5)
#     #plt.savefig('fits.png',dpi=100)
#     plt.show()
    
#     print(len(assoc))
#     mags = assoc['MAG_APER']
#     plt.hist(mags,bins='auto')
#     plt.show()
    return assoc

In [8]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'global color=lightgreen\nimage\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},5)\n'.format(xcoords[x],ycoords[x])
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
#         print(fileName)
#         print(os.listdir('.'))

In [9]:
def load_wcs_from_file(filename,pixCoord,ccd=0):
    # Load the FITS hdulist using astropy.io.fits
    hdulist = fits.open(filename)

    # Parse the WCS keywords in the primary HDU
    w = wcs.WCS(hdulist[ccd].header)

    # Print out the "name" of the WCS, as defined in the FITS header
    #print(w.wcs.name)

    # Print out all of the settings that were parsed from the header
    w.wcs.print_contents()

    # Three pixel coordinates of interest.
    # Note we've silently assumed a NAXIS=2 image here.
    # Note also that the pixel coordinates are pairs of [X, Y], 
    # and since WCS built from FITS header automatically has
    # origin set to 1, [0, 0] is actually not inside the image.
    pixcrd = np.array(pixCoord, dtype=np.float_)

    # Convert pixel coordinates to world coordinates
    # The second argument is "origin" -- in this case we're declaring we
    # have 1-based (Fortran-like) coordinates.
    world = w.wcs_pix2world(pixcrd, 1)
    #print(world)
    return(world)

In [10]:
def queryObjectRegion(outputTable,imageName,inCat,searchRadius=5*u.arcsecond,imDir='/home/mj1e16/simages/'):
    imageCoords = []
    r = searchRadius
    
    if inCat == 'dao':
        xlabel = 'xcentroid'
        ylabel = 'ycentroid'
        label = 'id'
        
    if inCat == 'sex':
        xlabel = 'X_IMAGE'
        ylabel = 'Y_IMAGE'
        label = 'NUMBER'
    flux = []   
    tableLength = len(outputTable)
    xpos = outputTable[xlabel]
    ypos = outputTable[ylabel]
    for x in range(tableLength):
        xpos = outputTable[xlabel][x]
        ypos = outputTable[ylabel][x]
        flux.append(outputTable['FLUX_MAX'][x])
        imageCoords.append([xpos,ypos])

    coords = load_wcs_from_file(imDir+imageName,imageCoords)
    customSimbad = Simbad()
    customSimbad.add_votable_fields('ra(d)','dec(d)','otype')
    customSimbad.remove_votable_fields('coordinates')
            
    result_table = Table([['-'],[0],[0],['-'],[0],[0],[0],['-']],names=('MAIN_ID','RA_d','DEC_d','OTYPE','xcentroid','ycentroid','flux','IMAGE'),dtype=('object','float64','float64','object','float64','float64','float64','object'))

    # change to update table and query in one go for the speed
#     for x in range(len(imageCoords)):
#         try:
#             c = SkyCoord(coords[x][0],coords[x][1],frame='icrs',unit='deg')
#             rt = customSimbad.query_region(c,radius=r)
#             rowlist = []
#             for y in range(len(rt[0])):
#                 rowlist.append(rt[0][y])
#             rowlist.append(imageCoords[x][0])
#             rowlist.append(imageCoords[x][1])
#             rowlist.append(flux[x])
#             rowlist.append(imageName)
#             result_table.add_row(rowlist)

#         except:
#             result_table.add_row(['N/A',[0],[0],'Object Not Found',[imageCoords[x][0]],[imageCoords[x][1]],flux[x],imageName])

   #result_table = Simbad.query_region(coord.SkyCoord
    for x in range(len(imageCoords)):
        try:
            c = SkyCoord(coords[x][0],coords[x][1],frame='icrs',unit='deg')
            rt = customSimbad.query_region(c,radius=r)
            rowlist = []
            for y in range(len(rt[0])):
                rowlist.append(rt[0][y])
            rowlist.append(imageCoords[x][0])
            rowlist.append(imageCoords[x][1])
            rowlist.append(flux[x])
            rowlist.append(imageName)
            result_table.add_row(rowlist)

        except:
            #print('searchfailed')
            result_table.add_row(['N/A',[0],[0],'Object Not Found',[imageCoords[x][0]],[imageCoords[x][1]],flux[x],imageName])

#     hdu_list = fits.open(imageName)
#     imagedata = (hdu_list[0].data) 
#     fig, ax = plt.subplots(figsize=(10, 10))
#     positions = (outputTable[xlabel], outputTable[ylabel])
#     apertures = CircularAperture(positions, r=4.)
#     norm = ImageNormalize(stretch=SqrtStretch())
#     ax.imshow(imagedata, cmap='Greys', origin='lower', vmin=-100,vmax=100)#norm=norm)
#     apertures.plot(color='blue', lw=1.5, alpha=0.5)
#     #plt.savefig('fits.png',dpi=100)
#     plt.show()
    
    return result_table

In [11]:
def runAndShow(image,cat,settings=[10,5,64,3],imDir='/home/mj1e16/Simages/',sexDir='/home/mj1e16/sextractor/sextractor-master/config/'):
    #alterDefault(sexDir,settings,['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE'],catName)
    tab = sextractor(sexDir,image,cat,imageDir=imDir)
    #print(tab)
    pan = tab.to_pandas()
    pan = pan[pan['MAG_BEST'] != 99.0]
    pan = pan[pan['ELLIPTICITY'] < np.median(pan['ELLIPTICITY'])]
    pan = pan[pan['ELONGATION'] < np.median(pan['ELONGATION'])]

#     pan = pan[pan['X_IMAGE'] < 610.]
#     pan = pan[pan['X_IMAGE'] > 570.]
#     pan = pan[pan['Y_IMAGE'] < 300.]
#     pan = pan[pan['Y_IMAGE'] > 250.]
    tab = Table.from_pandas(pan)
    print(len(tab))
    try:
        searchTab = queryObjectRegion(tab,image,'sex',imDir=imDir)
        return searchTab
    except:
        print('search failed')
        return('failed')
        pass
#     makeDS9RegFile([tab],'/home/mj1e16/simages/autosextest','sex')
#     subprocess.call(['ds9',imDir+image,'-regions','/home/mj1e16/simages/autosextest.reg'])
    

In [12]:
def smollBoi(image,imDir,cat=catName,sexDir='/home/mj1e16/sextractor/sextractor-master/config/',ccd='none'):
    tab = sextractor(sexDir,image,cat,imageDir=imDir)
    if len(tab) != 0:
        pan = tab.to_pandas()
        pan = pan[pan['MAG_BEST'] != 99.0]
#         pan = pan[pan['ELLIPTICITY'] < np.median(pan['ELLIPTICITY'])]
#         pan = pan[pan['ELONGATION'] < np.median(pan['ELONGATION'])]
        tab = Table.from_pandas(pan)
        xpos = tab['X_IMAGE']
        ypos = tab['Y_IMAGE']
        coords = [[x,y] for x,y in zip(xpos,ypos)]
        if ccd =='none':
            imageCoords = load_wcs_from_file(imDir+image,coords)
        else:
            imageCoords = load_wcs_from_file(imDir+image[:-3],coords,ccd=ccd)
        ra = [x[0] for x in imageCoords]
        dec = [x[1] for x in imageCoords]
        tab['RA'] = ra
        tab['DEC'] = dec
        names = findName(image)
        imname = names[0]
        medname = names[1]
        diffNames = [image]*len(ra)
        medNames = [medname]*len(ra)
        imNames = [imname]*len(ra)
        tab['IMAGE_NAME'] = imNames
        tab['MEDIAN_NAME'] = medNames
        tab['DIFF_NAME'] = diffNames
        #print(ra,dec)
    return tab

In [13]:
def mainImage(image,imDir,cat=catName,sexDir='/home/mj1e16/sextractor/sextractor-master/config/',ccd='none'):
    print(image)
    if ccd != 'none':
        imageplus = image+'[{}]'.format(ccd)
    tab = sextractor(sexDir,imageplus,cat,imageDir=imDir)
    if len(tab) != 0:
        pan = tab.to_pandas()
        pan = pan[pan['MAG_BEST'] != 99.0]
#         pan = pan[pan['ELLIPTICITY'] < np.median(pan['ELLIPTICITY'])]
#         pan = pan[pan['ELONGATION'] < np.median(pan['ELONGATION'])]
        tab = Table.from_pandas(pan)
        xpos = tab['X_IMAGE']
        ypos = tab['Y_IMAGE']
        coords = [[x,y] for x,y in zip(xpos,ypos)]
        if ccd =='none':
            imageCoords = load_wcs_from_file(imDir+image,coords)
        else:
            imageCoords = load_wcs_from_file(imDir+image,coords,ccd=ccd)
        ra = [x[0] for x in imageCoords]
        dec = [x[1] for x in imageCoords]
        tab['RA'] = ra
        tab['DEC'] = dec
        imNames = [image]*len(ra)
        tab['IMAGE_NAME'] = imNames

    return tab

In [14]:
def findName(image,diffDir=diffDir,medianDir=medianDir):
    hdu = fits.open(diffDir+image)
    head = hdu[0].header['DIFFCMD']
    locator1 = '/data/mj1e16/kepler/cal/'
    locator2 = ']'
    loc1 = head.find(locator1) + len(locator1)
    loc2 = head.find(locator2) + 1
    imname = head[loc1:loc2]
    
    channellocator1 = 'channel' 
    channellocator2 = '_ccd'
    ccdlocator1 = 'ccd'
    ccdlocator2 = '.fits'
    loc1 = image.find(channellocator1) + len(channellocator1)
    loc2 = image.find(channellocator2)
    channel = image[loc1:loc2]
    loc1 = image.find(ccdlocator1) + len(ccdlocator1)
    loc2 = image.find(ccdlocator2)
    ccd = image[loc1:loc2]
    
    medianName = 'meidian_{}_{}.fits'.format(channel,ccd)
    results = [imname,medianName]
    return results

In [15]:
def sexAll(dirlist,directory,sexDir=sexDir):
    os.chdir(sexDir)
    t = time.time()
    length = len(dirlist)
    section = 1
    for x in range(len(dirlist)):
        for ccd in range(1,85):
            totTab = smollBoi(dirlist[x]+'[{}]'.format(ccd),directory,ccd=ccd)
            ascii.write(totTab,'/home/mj1e16/outTablesImproved/imagecsvFinal/'+dirlist[x]+'[{}].csv'.format(ccd),format='csv',fast_writer=False)

    tend = time.time()
    print(tend-t)

In [18]:
sneIms = ['kplr2012242195726_ffi-cal.fits[45]','kplr2011271191331_ffi-cal.fits[1]','kplr2012004204112_ffi-cal.fits[44]']
sneDiffs = ['diff_imNo46_channel37_ccd45.fits','diff_imNo6_channel81_ccd1.fits','diff_imNo41_channel41_ccd44.fits']
ra = [293.3754458333333, 290.15641666666664, 291.1920374999999]
dec = [45.25035833333333, 38.252316666666665, 43.68074444444444]
xpos = [764.52512]
ypos = [331.56801]

In [28]:
atNames = ['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME']
valList = [10,6,'tophat_2.0_3x3.conv']
makeConfig(valList,atNames)

('default_test.sex', 'table', 'test_test.cat')

In [59]:
#def sextractor(sextractoryDir,imagename,cataloguename,imageDir='/home/mj1e16/keplerImages/'):
#def alterDefault(defaultDir,valList,attributeList,catName):


In [43]:
# os.chdir(sexDir)
# atNames = ['DETECT_THRESH','DETECT_MINAREA','FILER_NAME']
# valList = [2,2,'gauss_3.0_5x5.conv']
# #alterDefault(sexDir,valList,atNames,'test_test.cat',defaultname='default_test.sex',defaultnameOut='default_test.sex')
# outTab = sextractor(sexDir,sneDiffs[0],'test_test.cat',imageDir=diffDir)
# makeDS9RegFile([outTab],'superReg','sex')
# with open('superReg.reg','a') as f:
#     f.write('circle({},{},25)\n'.format(xpos[0],ypos[0]))
# subprocess.call(['ds9',diffDir+sneDiffs[0],'-regions','superReg.reg','-scale','mode','zscale'])

0

In [33]:
os.chdir(sexDir)
dirlist = os.listdir(imageDir)
print(len(dirlist))
dirlist = [x for x in dirlist if '.fits' in x]
dirlist = [x for x in dirlist if x[0] != '.']

print(len(dirlist))

for x in range(len(dirlist)):
    print(x)
    for ccd in range(1,85):
        imageplus = dirlist[x]+'[{}]'.format(ccd)
        outTab = mainImage(dirlist[x],imageDir,ccd=ccd)
        ascii.write(outTab,'/home/mj1e16/outTablesImproved/imagecsvFinal/'+imageplus+'.csv',format='csv',fast_writer=False,overwrite=True)

54
53
0
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits
kplr2012121122500_ffi-cal.fits


kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010111125026_ffi-cal.fits
kplr2010

kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011271191331_ffi-cal.fits
kplr2011

kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010265195356_ffi-cal.fits
kplr2010

kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011303191211_ffi-cal.fits
kplr2011

...

kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
kplr2009115002613_ffi-cal.fits
38
kplr2009116035924_ffi-cal.fits
kplr2009116035924_ffi-cal.fits
kplr2009116035924_ffi-cal.fits
kplr2009116035924_ffi-cal.fits
kplr2009116035924_ffi-cal.fits
kplr2009116035924_ffi-cal.fits
kplr2009116035924_ffi-cal.fits
kplr2009116035924_ffi-cal.fits
kplr2009116035924_ffi-cal.fits
kplr2009116035924_ffi-cal.fits
kplr2

kplr2010203012215_ffi-cal.fits
kplr2010203012215_ffi-cal.fits
kplr2010203012215_ffi-cal.fits
kplr2010203012215_ffi-cal.fits
kplr2010203012215_ffi-cal.fits
kplr2010203012215_ffi-cal.fits
kplr2010203012215_ffi-cal.fits
kplr2010203012215_ffi-cal.fits
kplr2010203012215_ffi-cal.fits
kplr2010203012215_ffi-cal.fits
41
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2012004204112_ffi-cal.fits
kplr2

kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010356020128_ffi-cal.fits
kplr2010

kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013065115251_ffi-cal.fits
kplr2013

kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012032101442_ffi-cal.fits
kplr2012

In [16]:
dirlist = os.listdir(diffDir)
print(len(dirlist))
dirlist = [x for x in dirlist if '.fits' in x]
dirlist = [x for x in dirlist if x[0] != '.']
print(len(dirlist))

os.chdir(sexDir)
t = time.time()
totTab = []

for x in range(len(dirlist)):
    if len(totTab) == 0:
        totTab = smollBoi(dirlist[x],diffDir)
    elif x == 2200:
        ascii.write(totTab,'/home/mj1e16/outTablesImproved/final/firsthalf.csv',format='csv',fast_writer=False)
        totTab = smollBoi(dirlist[x],diffDir)
    else:
        outTab = smollBoi(dirlist[x],diffDir)
        if outTab != 'failed':
            totTab = vstack([totTab,outTab])
    if x%100 ==0:
        percentage = x/4453.
        print('{}% complete'.format(percentage))
        print('runtime = {}'.format(time.time()-t))
    
print(time.time() -t)
ascii.write(totTab,'/home/mj1e16/outTablesImproved/final/secondhalf.csv',format='csv',fast_writer=False)

4452
4452
0.0% complete
runtime = 0.715242147446


/home/mj1e16/.local/lib/python2.7/site-packages/astropy/table/table.py:2628: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other


0.0224567707164% complete
runtime = 38.8054010868
0.0449135414327% complete
runtime = 90.8137001991
0.0673703121491% complete
runtime = 156.966860056
0.0898270828655% complete
runtime = 236.283905983
0.112283853582% complete
runtime = 329.542239189
0.134740624298% complete
runtime = 440.399960995
0.157197395015% complete
runtime = 563.874727011
0.179654165731% complete
runtime = 700.800957203
0.202110936447% complete
runtime = 848.658756971
0.224567707164% complete
runtime = 1010.04423809
0.24702447788% complete
runtime = 1184.04466701
0.269481248596% complete
runtime = 1373.34162116
0.291938019313% complete
runtime = 1576.62486506
0.314394790029% complete
runtime = 1793.60797215
0.336851560746% complete
runtime = 2025.49118018
0.359308331462% complete
runtime = 2272.761765
0.381765102178% complete
runtime = 2530.86093116
0.404221872895% complete
runtime = 2804.26742411
0.426678643611% complete
runtime = 3089.24137807
0.449135414327% complete
runtime = 3389.48961711
0.471592185044% com

In [12]:
#tabTot =  ascii.read('/home/mj1e16/sextractor/sextractor-master/config/sexdiffOutNew.csv')

In [32]:
ascii.write(totTab,'/home/mj1e16/outTablesImproved/final/firsthalf.csv',format='csv',fast_writer=False)

KeyboardInterrupt: 

In [14]:
#bigLength = len(tabTot)
bigLength = len(totTab)

In [18]:
redos = []
segs = 1000
customSimbad.add_votable_fields('ra(d)','dec(d)','otype')

for x in range(segs):
    start = int(x*(bigLength/segs))
    end = int((x+1)*(bigLength/segs))
    smollTab = tabTot[start:end]
    RA = smollTab['RA']
    DEC = smollTab['DEC']
    if x == 0:
        result_table = Simbad.query_region(coord.SkyCoord(ra=RA,dec=DEC,unit=(u.deg,u.deg),frame='fk5'),radius=5*u.arcsec)
    else:
        #try:
        newtab = Simbad.query_region(coord.SkyCoord(ra=RA,dec=DEC,unit=(u.deg,u.deg),frame='fk5'),radius=5*u.arcsec)
        result_table = vstack([result_table,newtab])
        print('Sucessful',x)
#         except ConnectionError:
#             print('Failed',x)
#             redos.append(x)
print('Sucessful: {}'.format(segs-x))
print('Failed: {}'.format(x))

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '19:15:13.9971 +41:34:20.0765': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 8 raised an error (recorded in the `errors` attribute of the result table): '19:21:43.2615 +41:05:41.2633': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 11 raised an error (recorded in the `errors` attribute of the result table): '19:21:17.0396 +41:03:27.2523': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-pack

('Sucessful', 1)

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '18:42:46.2101 +44:04:52.6381': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 5 raised an error (recorded in the `errors` attribute of the result table): '18:44:53.4454 +43:34:51.3244': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 9 raised an error (recorded in the `errors` attribute of the result table): '18:44:47.7093 +43:35:09.1375': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packa


('Sucessful', 2)


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '18:47:10.6785 +47:09:55.6909': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 5 raised an error (recorded in the `errors` attribute of the result table): '18:48:29.2053 +47:18:31.3264': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 8 raised an error (recorded in the `errors` attribute of the result table): '18:44:40.1278 +46:47:26.4974': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packa

...

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 5 raised an error (recorded in the `errors` attribute of the result table): '19:28:14.7934 +46:19:48.9184': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 8 raised an error (recorded in the `errors` attribute of the result table): '19:29:15.6158 +46:01:52.5111': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 10 raised an error (recorded in the `errors` attribute of the result table): '19:29:25.1181 +45:57:17.9373': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-pack

('Sucessful', 6)


('Sucessful', 7)


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 4 raised an error (recorded in the `errors` attribute of the result table): '19:38:29.1156 +46:14:10.2187': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 9 raised an error (recorded in the `errors` attribute of the result table): '19:36:50.5062 +46:47:59.8728': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 12 raised an error (recorded in the `errors` attribute of the result table): '19:36:10.2231 +47:02:40.4141': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-pack

MemoryError: 

In [18]:
ascii.write(totTab,'sexdiffOut.csv',format='csv',fast_writer=False)

In [1]:
dirlist = os.listdir(imdir)
dirlist = [x for x in dirlist if x[-4:] =='fits'] # check
#print(dirlist)


for im in range(len(dirlist)):
    if im == 0:
        outTab = runAndShow(dirlist[im],catName,imDir=imdir)
    else:
        newTab = runAndShow(dirlist[im],catName,imDir=imdir)
        try:
            outTab = vstack([outTab,newTab])
        except:
            print('No transients found in Image '+dirlist[im])
df = outTab.to_pandas()
df = df[df['OTYPE'] == 'Object Not Found']
dfsort = df.sort_values(['FLUX_MAX'])